# 2020 TO 2023 US Congress members' stock transactions analysis

## Table of Contents
- [Introduction and Posing Questions](#intro)
- [Data Collection and Wrangling](#wrangle)

<a id="intro"></a>
## Introduction and Posing Questions

In this project, I will perform exploratory data analysis on the data provided by [House Stock Watcher](https://housestockwatcher.com/api) on the US Congress members' stock transactions for the years 2020, 2021, 2022, and 2023. The dataset includes various columns providing information on disclosure dates, transaction dates, asset details, transaction types, amounts, and other relevant attributes.
I seek to uncover patterns, trends, and potential conflicts of interest within the data. As a result, I have developed thought-provoking questions that can guide my analysis and uncover meaningful insights:

1. What are the primary motivations driving Congress members' stock transactions?
   - How do demographic factors such as party affiliation, district representation, and state influence trading activities?
   - Are there noticeable patterns in trading behavior based on transaction types such as purchases or sales?
   - Can we identify any correlation such as transaction amount and party affiliations?  
2. Are there observable trends in the frequency and volumes of stock transactions over multiple years?
   - Do certain years exhibit higher trading activities among Congress members, and if so, what factors might contribute to these fluctuations?
   - How does transaction volume vary across different sectors and industries, are there sectors that Congress members show a particular interest in?
   - Can we identify any anomalies or spikes in trading activities warrant further investigation? 
3. Can we analyze the performance of Congress members' stock portfolios and identify any notable trends?
   - How do capital gain from stock transactions vary across different party affiliations?
   - Are there sectors that Congress members consistently realize higher capital gains, and what factors might explain these trends?
4. How can we cross-analyze trends between data dimensions to uncover deeper insights?
   - Are there correlations between transaction frequencies and demographic factors such as party affiliations or district representation?
   - How do sector-specific trends in stock transactions correlate with party affiliations?
   - Can we identify any systematic biases or patterns in trading activities that warrant further investigations?
     

<a id="wrangle"></a>
## Data Collection and Wrangling

The data utilized in this project is sourced from [House Stock Watcher](https://housestockwatcher.com), offering comprehensive stock transaction information of US Congress members spanning the years 2020 to 2023. Accessible via [House Stock Watcher API](https://house-stock-watcher-data.s3-us-west-2.amazonaws.com/data/all_transactions.json), this data is conveniently available in JSON format, facilitating integration and analysis.

In [1]:
## Import the necessary packages
import requests
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
## Get the data from the API and change it into Pandas DataFrame
api_url = "https://house-stock-watcher-data.s3-us-west-2.amazonaws.com/data/all_transactions.json"
response = requests.get(api_url)
data = response.json()
df = pd.DataFrame(data)
df.sample(3)

,disclosure_year,disclosure_date,transaction_date,owner,ticker,asset_description,type,amount,representative,district,state,ptr_link,cap_gains_over_200_usd,industry,sector,party
16658,2022,09/08/2022,2022-08-19,joint,GOGL,Golden Ocean Group Limited,purchase,"$1,001 - $15,000",Virginia Foxx,NC05,NC,https://disclosures-clerk.house.gov/public_dis...,False,Marine Transportation,Consumer Discretionary,Republican
15963,2022,03/23/2022,2022-03-09,joint,TMUS,T-Mobile US Inc,purchase,"$1,001 - $15,000",John Curtis,UT03,UT,https://disclosures-clerk.house.gov/public_dis...,False,Telecommunications Equipment,Public Utilities,Republican
3713,2022,02/07/2022,2022-01-05,None,OWLSX,Old Westbury Large Cap Strategies Fund,sale_full,"$15,001 - $50,000",Kurt Schrader,OR05,OR,https://disclosures-clerk.house.gov/public_dis...,False,None,None,Democrat
